In [1]:

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from scipy import stats
from scipy.interpolate import griddata

In [2]:
def crop_da(ds,min_lon,max_lon,min_lat,max_lat):

    mask_lon = (ds.lon >= min_lon) & (ds.lon <= max_lon)
    mask_lat = (ds.lat >= min_lat) & (ds.lat <= max_lat)

    ds = ds.where(mask_lon & mask_lat, drop=True)
    return ds

In [3]:
def rgb_to_grayscale(da):
    """Converts an RGB xarray DataArray to grayscale."""
    return (da.sel(rgb='red') * 0.2989 + 
            da.sel(rgb='green') * 0.5870 + 
            da.sel(rgb='blue') * 0.1140)

In [4]:


def remapping_3d_fields(xr_input,vars, output_lons, output_lats,output_times=None):

    input_lons = xr_input['lon'].values
    input_lats = xr_input['lat'].values
    input_times = xr_input['time'].values

    if output_times is None:
        output_times = input_times

    if len(input_lons.shape)==1:
        input_lons3, input_lats3= np.meshgrid(input_lons, input_lats)
    elif len(input_lons.shape)==2:

        input_lons3, input_lats3= np.meshgrid(input_lons[0,:], input_lats[0,:])
        # input_lons3 = np.repeat(input_lons[:,None,:], input_lats.shape[-1],axis=1)
        # input_lats3 = np.repeat(input_lats[:,:,None], input_lons.shape[-1],axis=2)
        # input_times3 = np.repeat(input_times[:,None], input_lons.shape[1],axis=1)
        # input_times3 = np.repeat(input_times3[:,:,None], input_lons.shape[2],axis=2)
    elif len(input_lons.shape)==3:
        input_lons3 = input_lons[0,...]
        input_lats3 = input_lats[0,...]
        # input_times3 = np.repeat(input_times[:,None], input_lons.shape[1],axis=1)
        # input_times3 = np.repeat(input_times3[:,:,None], input_lons.shape[2],axis=2)
    
    if len(output_lons.shape)==1:
        output_lons3, output_lats3 = np.meshgrid(output_lons, output_lats)
    elif len(output_lons.shape)==2:
        output_lons3, output_lats3= np.meshgrid(output_lons[0,:], output_lats[0,:])
        # output_lons3 = np.repeat(output_lons[:,None,:], output_lats.shape[-1],axis=1)
        # output_lats3 = np.repeat(output_lats[:,:,None], output_lons.shape[-1],axis=2)
        # output_times3 = np.repeat(output_times[:,  None], output_lons.shape[1],axis=1)
        # output_times3 = np.repeat(output_times3[:,:,None], output_lons.shape[2],axis=2)
    elif len(output_lons.shape)==3:
        output_lons3 = output_lons[0,...]
        output_lats3 = output_lats[0,...]
        # output_times3 = np.repeat(output_times[:,  None], output_lons.shape[1],axis=1)
        # output_times3 = np.repeat(output_times3[:,:,None], output_lons.shape[2],axis=2)
    
    input_lons3 = input_lons3.flatten()
    input_lats3 = input_lats3.flatten()
    # input_times3 = input_times3.flatten()
    # output_lons3 = output_lons3.flatten()
    # output_lats3 = output_lats3.flatten()
    # output_times3 = output_times3.flatten()

    input_points  = np.vstack((input_lons3 , input_lats3 )).T
    output_points = (output_lons3, output_lats3)
    xr_output = xr.Dataset({})
    # xr_output = xr.Dataset({'time': output_times, 'latitude': output_lats, 'longitude': output_lons})

    
    for var in vars:
        print(var)
        output_field = np.zeros((len(input_times),output_lons3.shape[0],output_lons3.shape[1]))
        for itime in range(len(input_times)):
            print('Time:', output_times[itime])
            input_field = xr_input[var].isel(time=itime).values
            output_field[itime,...] = griddata(input_points, input_field.flatten(), output_points, method='linear')
        xr_output[var] = xr.DataArray(output_field, dims=['time','latitude', 'longitude'], coords={'time': output_times,'latitude': output_lats, 'longitude': output_lons})
    return xr_output
    

In [5]:
direc_data = '../full_data/ai_ready/'

In [6]:
# Define the region of interest

min_lon = -125
max_lon = -114
min_lat = 32
max_lat = 42

In [7]:
# Load the datasets
ds_ci = xr.open_dataset(f'{direc_data}CloudImageryDataset.nc')
ds_co = xr.open_dataset(f'{direc_data}CloudOpticalDepthDataset.nc')
ds_sw = xr.open_dataset(f'{direc_data}ReflectedSWDataset.nc')
# Convert the RGB images to grayscale
ds_ci['GrayImagery'] = rgb_to_grayscale(ds_ci.CloudImagery)


In [9]:
# Crop the datasets
ds_ci = crop_da(ds_ci,min_lon,max_lon,min_lat,max_lat)
ds_co = crop_da(ds_co,min_lon,max_lon,min_lat,max_lat)
ds_sw = crop_da(ds_sw,min_lon,max_lon,min_lat,max_lat)

In [10]:
# Reverse the latitude dimension to have it in ascending order

ds_ci=ds_ci.isel(latitude=slice(None, None, -1))
ds_co=ds_co.isel(latitude=slice(None, None, -1))
ds_sw=ds_sw.isel(latitude=slice(None, None, -1))

In [12]:
lon_remapped = np.linspace(-125,-114,11*40+1)
lat_remapped = np.linspace(32,42,10*40+1)


In [13]:
ds_co_remapped = remapping_3d_fields(ds_co, ['CloudOpticalDepth'], lon_remapped, lat_remapped,None)
ds_ci_remapped = remapping_3d_fields(ds_ci, ['GrayImagery'], lon_remapped, lat_remapped,None)
ds_sw_remapped = remapping_3d_fields(ds_sw, ['ReflectedSW'], lon_remapped, lat_remapped,None)

CloudOpticalDepth
0
1
2
3
4
5
6
7
8
9
GrayImagery
0
1
2
3
4
5
6
7
8
9
ReflectedSW
0
1
2
3
4
5
6
7
8
9
